In [1]:
#-*- coding: utf-8 -*-
from google.cloud import bigquery
from datetime import timedelta
from datetime import datetime
import argparse
import time
import sys, os
from time import sleep 

In [2]:
# Real배포시 수정사항
# 1. os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/mi_admin/miinfra-project-90dbee12067b.json"
# 2. parse_parameters(): 안에 막는부분과 푸는 부분


os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/mi_admin/miinfra-project-90dbee12067b.json"


In [3]:
def get_timedelta(sdatetime, edatetime):
    turnaround = edatetime - sdatetime
    total_seconds = int(turnaround.total_seconds())
    hours, remainder = divmod(total_seconds,60*60)
    minutes, seconds = divmod(remainder,60)
    out = '{:02}:{:02}:{:02})'.format(hours,minutes,seconds)
    return out

In [4]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [5]:
def query_log(log_file_name, query_string):
    sql_log_filename = query_log_path+'/'+log_file_name
    with open(sql_log_filename, 'w') as file:
        file.write(query_string)
        file.close()


In [6]:
def get_totalrows(schema_id, table_id):
    start_now = datetime.now()
    query_string = """Select Count(*) total_rows
  From """+schema_id+"""."""+table_id
    client = bigquery.Client(project='miinfra-project')
    query_job = client.query(query_string)
    total_rows = [row['total_rows'] for row in query_job][0]
    #print('Execute >> [start:{}][elapsed:{}][{: 10} rows][row selected] [{}.{}]'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), get_timedelta(start_now, datetime.now()), total_rows, schema_id, table_id))
    return total_rows

In [7]:
def query_execute_create(destination_schema, destination_table, query_string, temp_table_name, step_name):
    start_now = datetime.now()
    # Query 내용 로그처리
    log_file_name = temp_table_name+'_'+step_name+'.sql'
    query_log(log_file_name, query_string)

    client = bigquery.Client(project='miinfra-project')
    job_config = bigquery.QueryJobConfig()

    dataset_id=destination_schema
    table_id = destination_table
    table_ref = client.dataset(dataset_id).table(table_id)
    job_config.destination = table_ref
    job_config.write_disposition = 'WRITE_TRUNCATE'
    query_job = client.query(query_string, job_config=job_config)
    results = query_job.result()  
    # 1번째 방법
    #df_result = results.to_dataframe()
    #rows = results.total_rows
    # 2번째 방법
    # rows = len([i[0] for i in results])
    #rows = 0
    # 3번째 방법
    rows = get_totalrows(dataset_id, table_id)
    print('Execute >> [start:{}][elapsed:{}][{: 10} rows][{}] [{}.{}]'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), get_timedelta(start_now, datetime.now()), rows, step_name, destination_schema, destination_table))

    

In [8]:
def query_execute_dml(query_string, temp_table_name, step_name):
    start_now = datetime.now()
    # Query 내용 로그처리
    log_file_name = temp_table_name+'_'+step_name+'.sql'
    query_log(log_file_name, query_string)

    client = bigquery.Client(project='miinfra-project')
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query_string, job_config=job_config)
    results = query_job.result()  
    rows = query_job.num_dml_affected_rows
    print('Execute >> [start:{}][elapsed:{}][{: 10} rows][{}]'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), 
                                                            get_timedelta(start_now, datetime.now()), rows, step_name))
    #print(results)

In [9]:
def globalad_exist(date_key):
    date_key_nodash = date_key.replace('-', '')
    query_string = """Select Count(*) as onoff From Tier0_BI.__TABLES__ Where table_id='Log_GlobalAd_"""+date_key_nodash+"""' """
    print(query_string)
    client = bigquery.Client(project='miinfra-project')
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query_string, job_config=job_config)
    results = query_job.result()  
    for i in results:
        rows = i[0]
    return rows

In [10]:
def get_campaignid(step_name):
    start_now = datetime.now()
    query_string = '''Select IFNULL(Max(CampaignID), 1000000) as campaignid
  From Tier1_BI.code_info_campaign_ver3_develop
 Where CampaignID > 1000000 
    '''
    client = bigquery.Client(project='miinfra-project')
    query_job = client.query(query_string)
    campaignid_cnt = [row['campaignid'] for row in query_job][0]
    print('Execute >> [start:{}][elapsed:{}]                 [{}] (campaignid max : {})'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), get_timedelta(start_now, datetime.now()),step_name,campaignid_cnt))
    return campaignid_cnt

In [11]:
def get_channelid(step_name):
    start_now = datetime.now()
    query_string = '''Select IFNULL(Max(ChannelID), 1000000) as channelid
  From Tier1_BI.code_info_channel_ver3_develop
 Where ChannelID > 1000000 
    '''
    client = bigquery.Client(project='miinfra-project')
    query_job = client.query(query_string)
    channelid_cnt = [row['channelid'] for row in query_job][0]
    print('Execute >> [start:{}][elapsed:{}]                 [{}] (channelid max : {})'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), get_timedelta(start_now, datetime.now()),step_name,channelid_cnt))
    return channelid_cnt

In [12]:
def get_trafficsourceid(step_name):
    start_now = datetime.now()
    query_string = '''Select IFNULL(Max(TrafficSourceID), 1000000) as trafficsourceid
  From Tier1_BI.code_info_trafficsource_ver3_develop
 Where TrafficSourceID > 1000000 
    '''
    client = bigquery.Client(project='miinfra-project')
    query_job = client.query(query_string)
    trafficsourceid_cnt = [row['trafficsourceid'] for row in query_job][0]
    print('Execute >> [start:{}][elapsed:{}]                 [{}] (trafficsourceid max : {})'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), get_timedelta(start_now, datetime.now()),step_name,trafficsourceid_cnt))
    return trafficsourceid_cnt

In [13]:
def get_adid(step_name):
    start_now = datetime.now()
    query_string = '''Select IFNULL(Max(ADID), 1000000) as adid
  From Tier1_BI.code_info_ad_ver3_develop
 Where ADID > 1000000 
    '''
    client = bigquery.Client(project='miinfra-project')
    query_job = client.query(query_string)
    adid_cnt = [row['adid'] for row in query_job][0]
    print('Execute >> [start:{}][elapsed:{}]                 [{}] (adid max : {})'.format(datetime.strftime(start_now, '%Y-%m-%d %H:%M:%S'), get_timedelta(start_now, datetime.now()),step_name,adid_cnt))
    return adid_cnt

In [14]:
def query_execute_df(query_string):
    client = bigquery.Client(project='miinfra-project')
    query_job = client.query(query_string)

    return query_job.to_dataframe()

In [15]:
def step_01_source_data(date_key_dt):
    date_key_yyyymmdd = date_key_dt.strftime('%Y%m%d')
    query_string = ''
    
    ###################################################################
    # Pub Default
    ###################################################################
    pub_twitter = 'publisher_sub_campaign_name'
    pub_unity_ads = 'sub_site'
    pub_google_adwords = 'sub_campaign'
    pub_adscreen = 'sub_publisher'
    pub_cheetah_mobile = 'publisher_sub_site'
    pub_cashslide_ncpi = 'publisher_sub1'
    pub_cashslide = 'publisher_sub1'
    pub_glispa = 'sub_site'
    pub_tapjoy = 'Publisher_sub_site'
    pub_vungle = 'Publisher_sub_site'
    pub_chartboost = 'publisher_sub_site_name'
    pub_fyber_formerly_sponsorpay = 'sub_site'
    pub_adcolony = '''Concat(sub_publisher,'>',publisher_sub_site_name)'''
    pub_the_trade_desk = 'my_adgroup'
    pub_crossinstall = 'sub_site'
    pub_snapchat = 'publisher_sub_campaign_name'
    pub_adeals = 'publisher_sub_site_name'
    pub_105 = 'sub_publisher'
    
    
    if date_key_dt <= datetime.strptime('2018-09-07', '%Y-%m-%d'):
        pub_adeals = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2018-03-21', '%Y-%m-%d'):
        pub_vungle = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2018-03-15', '%Y-%m-%d'):
        pub_tapjoy = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2017-11-14', '%Y-%m-%d'):
        pub_snapchat = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2017-11-12', '%Y-%m-%d'):
        pub_crossinstall = 'sub_publisher'        
    if date_key_dt <= datetime.strptime('2017-09-05', '%Y-%m-%d'):
        pub_adcolony = 'sub_publisher'
        pub_the_trade_desk = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2017-07-29', '%Y-%m-%d'):
        pub_fyber_formerly_sponsorpay = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2017-07-15', '%Y-%m-%d'):
        pub_adscreen = 'publisher_sub4'
        pub_chartboost = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2017-07-09', '%Y-%m-%d'):
        pub_glispa = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2017-03-12', '%Y-%m-%d'):
        pub_adscreen = 'sub_publisher'
        pub_cheetah_mobile = 'sub_publisher'
        pub_cashslide_ncpi = 'sub_publisher'
        pub_cashslide = 'sub_publisher'
    if date_key_dt <= datetime.strptime('2016-09-26', '%Y-%m-%d'):
        pub_twitter = 'sub_site'
    if date_key_dt <= datetime.strptime('2016-08-06', '%Y-%m-%d'):
        pub_twitter = 'publisher_sub_campaign_name'
        pub_unity_ads = 'publisher_sub_publisher_name'
        pub_google_adwords = 'publisher_sub_publisher_name'
        pub_adcolony = 'publisher_sub_publisher_name'
        pub_the_trade_desk = 'publisher_sub_publisher_name'
        pub_105 = 'publisher_sub_publisher_name'
        

    ###################################################################
    # Subpub Default
    ###################################################################
    subpub_twitter = 'sub_site'
    subpub_google_adwords = 'sub_site'
    subpub_chartboost = 'publisher_sub_campaign_name'
    subpub_mobvista = 'publisher_sub_campaign_name'
    subpub_unity_ads = 'publisher_sub_campaign_name'
    subpub_vungle = 'publisher_sub_campaign_name'
    subpub_adscreen = ''' '' '''
    subpub_105 = 'my_campaign'

    if date_key_dt <= datetime.strptime('2017-01-01', '%Y-%m-%d'):
        subpub_adscreen = 'publisher_ref_id'        
    if date_key_dt <= datetime.strptime('2016-10-01', '%Y-%m-%d'):
        subpub_adscreen = 'publisher_sub_campaign_name'        
    if date_key_dt <= datetime.strptime('2016-09-26', '%Y-%m-%d'):
        subpub_twitter = 'publisher_sub_campaign_name'
    if date_key_dt <= datetime.strptime('2016-08-06', '%Y-%m-%d'):
        subpub_twitter = 'null'
        subpub_chartboost = 'publisher_sub_campaign_name'
        subpub_mobvista = 'publisher_sub_campaign_name'
        subpub_unity_ads = 'publisher_sub_campaign_name'
        subpub_vungle = 'publisher_sub_campaign_name'
        subpub_adscreen = 'publisher_sub_campaign_name'
        subpub_google_adwords = 'publisher_sub_campaign_name'
        subpub_105 = 'publisher_sub_campaign_name'
   
    query_string = '''    
Select gamecode, LogDetailID, Trim(CampaignName) as CampaignName, Trim(ChannelName) as ChannelName, Trim(ADName) as ADName, Min(regdatetime) regdatetime
  From (Select gamecode, LogDetailID, CampaignName, ChannelName, ADName, Min(regdatetime) regdatetime
        From (
              Select Upper(gamecode) as gamecode, Case When LogDetailID in (1,3) Then 1 When LogDetailID in (2,4) Then 2 Else 0 End LogDetailID,
                  IFNULL(Upper(Case When LogID=105 And Upper(publisher_name) = Upper('nanigans') Then Upper('Twitter')
                                    When LogID=105 And Upper(publisher_name) = Upper('Ironsource') Then Upper('Supersonic')
                                    When LogID=105 Then Upper(publisher_name)
                                    When LogID=7777 Then Upper('ORGANIC')
                                     When LogID=106 Then Case When Upper(network_name) Like '%FACEBOOK%' OR Upper(network_name) Like '%INSTAGRAM%'
                                                                                     Then Upper('FACEBOOK') Else Upper(network_name) End End),'ORGANIC') as CampaignName,
                  IFNULL(Upper(Case When LogID=105 And Upper(publisher_name) = Upper('Twitter') Then '''+pub_twitter+ '''
                            When LogID=105 And Upper(publisher_name) = Upper('Adcolony performance media') then sub_site    -- 20170220 추가
                            When LogID=105 And Upper(publisher_name) = Upper('Apple Search Ads')     Then publisher_sub_adgroup_name                 -- 20180426 추가
                            When LogID=105 And Upper(publisher_name) = Upper('nanigans') Then publisher_sub_campaign_name
                            When LogID=105 And Upper(publisher_name) = Upper('LoopMe') Then my_site
                            When LogID=105 And Upper(publisher_name) = Upper('unity ads') Then '''+pub_unity_ads+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Google Adwords') Then '''+pub_google_adwords+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Adscreen')       Then '''+pub_adscreen+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Cheetah Mobile') Then '''+pub_cheetah_mobile+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Cashslide_NCPI') Then '''+pub_cashslide_ncpi+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Cashslide')      Then '''+pub_cashslide+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Glispa')         Then '''+pub_glispa+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Tapjoy')         Then '''+pub_tapjoy+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Vungle')         Then '''+pub_vungle+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Chartboost' )    Then '''+pub_chartboost+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Fyber formerly SponsorPay')     Then '''+pub_fyber_formerly_sponsorpay+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Adcolony')       Then '''+pub_adcolony+'''
                            When LogID=105 And Upper(publisher_name) = Upper('The Trade Desk') Then '''+pub_the_trade_desk+'''
                            When LogID=105 And Upper(publisher_name) = Upper('CrossInstall')     Then '''+pub_crossinstall+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Snapchat')     Then '''+pub_snapchat+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Adeals')     Then '''+pub_adeals+'''
                            When LogID=105 Then '''+pub_105+'''
                            When LogID=106 And Upper(network_name) = Upper('apple search ads') Then site_id
                            When LogID=106 Then campaign_name
                            When LogID=7777 Then ADSet_Name End),'ORGANIC') as ChannelName,
                  IFNULL(Upper(Case When LogID=105 And Upper(publisher_name) = Upper('Twitter')  Then '''+subpub_twitter+'''
                                      When LogID=105 And Upper(publisher_name) = Upper('nanigans')  Then sub_site
                                      When LogID=105 And Upper(publisher_name) = Upper('Apple Search Ads')  Then publisher_sub_keyword_name 
                                      When LogID=105 And Upper(publisher_name) = Upper('chartboost') Then '''+subpub_chartboost+'''
                                      When LogID=105 And Upper(publisher_name) = Upper('mobvista') Then '''+subpub_mobvista+'''
                                      When LogID=105 And Upper(publisher_name) = Upper('unity ads') Then '''+subpub_unity_ads+'''
                                      When LogID=105 And Upper(publisher_name) = Upper('vungle') Then '''+subpub_vungle+'''
                                      When LogID=105 And Upper(publisher_name) = Upper('adscreen') Then '''+subpub_adscreen+'''
                            When LogID=105 And Upper(publisher_name) = Upper('Google Adwords') Then '''+subpub_google_adwords+'''
                            When LogID=105 Then '''+subpub_105+'''
                            When LogID=106 Then adgroup_name
                            When LogID=7777 Then Upper(AD_Name) End),'ORGANIC') as ADName,
                   Min(Timestamp(regdatetime)) as regdatetime
              From Tier0_BI.Log_GlobalAd_'''+date_key_yyyymmdd+''' a 
                Where ((a.LogID IN (105,7777) And (((Upper(publisher_name) not like '%FACEBOOK%') And (Upper(publisher_name) not like '%INSTAGRAM%')) or publisher_name is null ))
                                        OR (a.LogID = 106 And ((Upper(a.network_name) Like '%FACEBOOK%') OR (Upper(a.network_name) Like '%INSTAGRAM%'))))
                  And LogDetailID IN (1,2,3,4)
                  And (replay is Null OR replay = '')       -- 재전송인경우 영향을 받지 않기 위해서 추가(20180502)
              Group By gamecode, LogDetailID, CampaignName, ChannelName, ADName     ) Last
          Group By gamecode, LogDetailID, CampaignName, ChannelName, ADName
          Union All
          Select Upper(SchemaNM), b.LogDetailID, Upper('Organic'), Upper('Organic'), Upper('Organic'),  timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second)
            From Tier1_BI.code_dim_gamelist a 
              Cross Join (Select 1 as LogDetailID Union All Select 2) b
              ) c
      Group By gamecode, LogDetailID, CampaignName, ChannelName, ADName
                  
                  '''
    #  'テリアサ?ガ' 와 'てりあさ?が' MSSQL은 동일한 문자로 인식하고, BigQuery는 서로 다른 문자로 인식한다. 이로 인해 몇건 차이가 난다.

    return query_string

In [16]:
def step_02_01_query_string(source_schema, source_table):
    query_string = """Select gamecode, LogDetailID, CampaignName, Min(Regdatetime) as Regdatetime
  From """+source_schema+"""."""+source_table+"""
 Group By gamecode, LogDetailID, CampaignName
 """ 
    return query_string

In [17]:
def step_02_03_query_string(campaign_schema, campaign_table, campaignid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_campaign_ver3_develop
    (CampaignID, CampaignName, AppID, Status, Status_codekind, MediaType, MediaType_codekind, AdvertiseType, AdvertiseType_codekind, 
     TrafficType, TrafficType_codekind, StartDate, EndDate, TotalBudget, currency, currency_codekind, redirectUrl, webview_info, ExtractDT)
    Select """+str(campaignid)+""" + Row_Number() Over (Order By c.RegDatetime) as CampaignID, 
       Upper(c.CampaignName) CampaignName, App.AppID, 
       '01' as Status, '0006' as Status_codekind, 
       '105' as MediaType, '0017' as MediaType_codekind,
       '01' as AdvertiseType, '0004' as AdvertiseType_codekind, 
       '06' as TrafficType, '0005' as TrafficType_codekind,
       Timestamp('"""+date_key+"""') as StartDate, Timestamp('2050-01-01') as EndDate,
       0.0 as TotalBudget, '01' as currency, '0007' as currency_codekind, '' as redirectUrl, 
       '0000000000' as webview_info, timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT
  From Tier1_BI.code_info_app_ver3_develop App
       Inner Join Tier1_BI.code_dim_gamelist b ON Cast(App.gamekey as INT64) = b.gamekey
       Inner Join """+campaign_schema+"""."""+campaign_table+""" c 
              ON Case When Upper(b.schemaNM)=Upper('chacha') Then Upper('chachacha')
                      When Upper(b.schemaNM)=Upper('pongpong') Then Upper('everypong') Else Upper(b.schemaNM) End = Upper(c.gamecode) 
              And App.market = Case When LogDetailid = 1 Then '01' Else '02' End
       Left outer Join Tier1_BI.code_info_campaign_ver3_develop Campaign ON App.AppID = Campaign.AppID And Trim(Upper(Campaign.CampaignName)) = Trim(Upper(c.CampaignName))
 Where Campaign.AppID IS NULL
    """
    
    #print(query_string)
    
    return query_string

In [18]:
def step_02_05_query_string(campaignid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_campaign_ver3_develop
    (CampaignID, CampaignName, AppID, Status, Status_codekind, MediaType, MediaType_codekind, AdvertiseType, AdvertiseType_codekind, 
     TrafficType, TrafficType_codekind, StartDate, EndDate, TotalBudget, currency, currency_codekind, redirectUrl, webview_info, ExtractDT)
Select """+str(campaignid)+""" + Row_Number() Over (Order By GameKey) as CampaignID, 
       Upper('Netmarble Mass Marketing') as CampaignName, a.AppID,
       '01' as Status, '0006' as Status_codekind, 
       '1' as mediaType, '0017' as mediatype_codekind,
       '01' as AdvertiseType, '0004' as AdvertiseType_codekind,
       '07' as TrafficType, '0005' as TrafficType_codekind,
       Timestamp('"""+date_key+"""') as StartDate, Timestamp('2050-01-01') as EndDate,
       0.0 as TotalBudget, '04' as currency, '0007' as currency_codename, '' as redirectURL,
       '0000000000' as webview_info, timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT
  From Tier1_BI.code_info_app_ver3_develop a
       Left Outer Join Tier1_BI.code_info_campaign_ver3_develop b
            ON a.AppID = b.AppID And Trim(Upper(b.CampaignName)) = Trim(Upper('Netmarble Mass Marketing'))
 Where b.AppID IS NULL
    """
    
    #print(query_string)
    
    return query_string

In [19]:
def step_03_01_query_string(source_schema, source_table):
    query_string = """Select channelname
  From """+source_schema+"""."""+source_table+"""
 Group By channelname
 """
    return query_string

In [20]:
def step_03_03_query_string(channel_schema, channel_table, channelid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_channel_ver3_develop
    (ChannelID, ChannelName, ChannelType, ChannelType_codekind, Status, Status_codekind, PricingTypeArray, Adver_AppID, ExtractDT)
    Select """+str(channelid)+""" + Row_Number() Over (Order By a.ChannelName) as ChannelID, 
           a.ChannelName, 
           '03' as ChannelType, '0008' as ChannelType_codekind, 
           '01' as Status, '0009' as Status_codekind, 
           '' as PricingTypeArray, 0 as Adver_AppID, timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT
      From """+channel_schema+"""."""+channel_table+""" a
           Left Outer Join Tier1_BI.code_info_channel_ver3_develop b On Trim(Upper(a.ChannelName)) = Trim(Upper(b.ChannelName))
                And ChannelID > 1000000
     Where b.ChannelID IS NULL
    """
    
    #print(query_string)
    
    return query_string

In [21]:
def step_04_01_query_string(source_schema, source_table):
    query_string = """Select LogDetailID, gamecode, CampaignName, ChannelName
  From """+source_schema+"""."""+source_table+"""
 Group By LogDetailID, gamecode, CampaignName, ChannelName
 """
    return query_string

In [22]:
def step_04_03_query_string(trafficsource_schema, trafficsource_table, trafficsourceid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_trafficsource_ver3_develop
    (TrafficSourceID, CampaignID, ChannelID, Status, Status_codekind, TotalBudget, SpentBudget, Price, 
     currency, currency_codekind, PricingType, PricingType_codekind, AppID, webview_info, ExtractDT)
Select """+str(trafficsourceid)+""" + Row_Number() Over (Order By T.CampaignID, U.ChannelID) as TrafficSourceID, 
       T.CampaignID, U.ChannelID, 
       '01' as Status, '0010' as Status_codekind, 
       0 as TotalBudget, 0 as SpentBudget, 0 as Price,
       '01' as currency, '0007' as currency_codekind,
       '99' as PricingType, '0011' as PricingType_codekind,
       T.AppID, '0000000000' as webview, timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT
  From """+trafficsource_schema+"""."""+trafficsource_table+""" S
       Inner Join (Select CampaignID, MediaType, a.CampaignName, a.appid, b.market, 
                               Case When Upper(c.schemaNM)=Upper('chacha') Then Upper('chachacha')
                                    When Upper(c.schemaNM)=Upper('pongpong') Then Upper('everypong') Else Upper(c.schemaNM) End as Gamecode
                          From Tier1_BI.code_info_campaign_ver3_develop a 
                               Inner Join Tier1_BI.code_info_app_ver3_develop b  On a.appid = b.appid
                               Inner Join Tier1_BI.code_dim_gamelist c On Cast(b.gamekey as INT64) = c.gamekey
                         Where a.CampaignID > 1000000) T On Trim(Upper(S.CampaignName)) = Trim(Upper(T.CampaignName))
            And S.gamecode = T.Gamecode And Case When S.LogDetailID = 1 Then '01' Else '02' End = T.market 
       Inner Join Tier1_BI.code_info_channel_ver3_develop U ON Trim(Upper(S.ChannelName)) = Trim(Upper(U.ChannelName)) 
                  And U.ChannelID > 1000000
       Left Outer Join Tier1_BI.code_info_trafficsource_ver3_develop V ON T.CampaignID = V.CampaignID
                  And U.ChannelID = V.ChannelID 
                  And V.CampaignID > 1000000
                  And V.ChannelID > 1000000
                  And V.TrafficSourceID > 1000000
 Where V.TrafficSourceID IS NULL
    """
    #print(query_string)
    return query_string

In [23]:
def step_04_05_query_string(trafficsource_schema, trafficsource_table, trafficsourceid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_trafficsource_ver3_develop
    (TrafficSourceID, CampaignID, ChannelID, Status, Status_codekind, TotalBudget, SpentBudget, Price, 
     currency, currency_codekind, PricingType, PricingType_codekind, AppID, webview_info, ExtractDT)

Select """+str(trafficsourceid)+""" + Row_Number() Over (Order By AA.AppID, AA.CampaignID, AA.ChannelID) as TrafficSourceID, 
       AA.CampaignID, AA.ChannelID, '01' as Status,'0010' as Status_codekind, 0 as totalBudget, 0 as SpentBudget, 0 as Price, 
       '04' as currency, '0007' as currency_codekind, '99' as PricingType, '0011' as PricingType_codekind, 
       AA.AppID, '' webview_info, timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT
  From (Select AppID, CampaignID, ChannelID
          From Tier1_BI.code_info_campaign_ver3_develop a
               Inner Join Tier1_BI.code_info_channel_ver3_develop b 
                     ON Trim(Upper(b.ChannelName)) IN ('UA','TV','OOH','VIRAL','ETC','COMEBACKUA')
         Where Trim(Upper(a.CampaignName)) = Trim(Upper('Netmarble Mass Marketing')) ) AA
        Left Outer Join Tier1_BI.code_info_trafficsource_ver3_develop BB
             ON AA.CampaignID = BB.CampaignID And AA.ChannelID = BB.ChannelID
 Where BB.TrafficSourceID IS NULL

 
    """
    #print(query_string)
    return query_string

In [24]:
def step_04_07_query_string(trafficsource_schema, trafficsource_table, trafficsourceid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_trafficsource_ver3_develop
    (TrafficSourceID, CampaignID, ChannelID, Status, Status_codekind, TotalBudget, SpentBudget, Price, 
     currency, currency_codekind, PricingType, PricingType_codekind, AppID, webview_info, ExtractDT)
Select """+str(trafficsourceid)+""" + Row_Number() Over (Order By AA.AppID, AA.CampaignID, AA.ChannelID) as TrafficSourceID, 
       AA.CampaignID, AA.channelID, 
       '01' as Status,'0010' as Status_codekind, 0 as totalBudget, 0 as SpentBudget, 0 as Price, 
       '04' as currency, '0007' as currency_codekind, '99' as PricingType, '0011' as PricingType_codekind, 
       AA.AppID, '' webview_info, timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT
  From (
        Select a.AppID, a.CampaignID, b.ChannelID
          From Tier1_BI.code_info_campaign_ver3_develop a
               Inner Join Tier1_BI.code_info_channel_ver3_develop b
                     ON Upper(b.ChannelName) = '공통'
         Where Mediatype in ('105','7777') ) AA
       Left Outer Join Tier1_BI.code_info_trafficsource_ver3_develop BB
            ON AA.AppID = BB.appID And AA.CampaignID = BB.CampaignID And AA.ChannelID = BB.ChannelID
 Where BB.AppID IS NULL

 
    """
    #print(query_string)
    return query_string

In [25]:
def step_05_02_query_string(source_schema, source_table, adid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_ad_ver3_develop
    (ADID, AppID, CampaignID, ChannelID, Location, Status, Status_codekind, ADType, ADType_codekind, View_Order, 
     TotalBudget, SpentBudget, Price, webview_info, ExtractDT, trafficsourceid, Tracking_Link)
Select """+str(adid)+""" + Row_Number() Over (Order By a.CampaignID) as ADID, 
       a.AppID, a.CampaignID, a.ChannelID, f.ADName Location,
       '01' as Status, '0015' as Status_codekind,
       '99' as ADType, '0016' as ADType_codekind,
       1 as View_Order, 0 as TotalBudget, 0 as SpentBudget, 0 as price,
       '0000000000' as webview, timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT, a.TrafficSourceID, '' as Tracking_Link
  From Tier1_BI.code_info_trafficsource_ver3_develop a
       Inner Join Tier1_BI.code_info_channel_ver3_develop b On a.channelID = b.ChannelID
       Inner Join Tier1_BI.code_info_app_ver3_develop c On a.Appid = c.appid
       Inner Join Tier1_BI.code_dim_gamelist d on Cast(c.gamekey as INT64) = d.gamekey
       Inner Join Tier1_BI.code_info_campaign_ver3_develop e On a.CampaignID = e.CampaignID
       Inner Join """+source_schema+"""."""+source_table+""" f On Trim(Upper(e.CampaignName)) = Trim(Upper(f.CampaignName))
             And Upper(d.schemaNM) = Upper(f.gamecode) And Trim(Upper(b.ChannelName)) = Trim(Upper(f.ChannelName))
             And Case When f.LogDetailID = 1 Then '01' Else '02' End = c.market
       Left Join Tier1_BI.code_info_ad_ver3_develop g On a.AppID = g.AppID
            And a.CampaignID = g.CampaignID And a.ChannelID = g.ChannelID And a.TrafficSourceID = g.trafficsourceid
            And Trim(Upper(f.ADName)) = Trim(Upper(g.Location))
            And g.ADID > 1000000 
 Where a.TrafficSourceID > 1000000 
   And g.AppID IS NULL
 order by 1,2,3

 
    """
    #print(query_string)
    return query_string

In [26]:
def step_05_04_query_string(ad_schema, ad_table, adid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_ad_ver3_develop
    (ADID, AppID, CampaignID, ChannelID, Location, Status, Status_codekind, ADType, ADType_codekind, View_Order, 
     TotalBudget, SpentBudget, Price, webview_info, ExtractDT, trafficsourceid, Tracking_Link)

Select """+str(adid)+""" + Row_Number() Over (Order By a.TrafficsourceID) as ADID, 
       a.AppID, a.CampaignID, a.ChannelID, c.ChannelName Location,
       '01' Status,'0015' Status_codekind, '99' as ADType, '0016' as ADType_codekind,
       1 as View_Order, 0 TotalBudget, 0 as SpentBudget, 0 as Price, '' as webview_info,
       timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT, a.TrafficSourceID, Tracking_Link
  From Tier1_BI.code_info_trafficsource_ver3_develop a
       Inner Join Tier1_BI.code_info_campaign_ver3_develop b
             ON a.campaignID = b.campaignID And Trim(Upper(b.CampaignName)) = Trim(Upper('Netmarble Mass Marketing'))
       Inner Join Tier1_BI.code_info_channel_ver3_develop c
             ON a.ChannelID = c.ChannelID
       Left outer Join Tier1_BI.code_info_ad_ver3_develop d
            ON a.AppID = d.AppID and a.CampaignID = d.CampaignID And a.ChannelID = d.ChannelID And a.TrafficSourceID = d.trafficsourceid
            And Trim(Upper(d.Location)) = Trim(Upper(c.ChannelName))
 Where d.AppID IS NULL

 
    """
    #print(query_string)
    return query_string

In [27]:
def step_05_06_query_string(ad_schema, ad_table, adid, date_key):
    query_string = """Insert Into Tier1_BI.code_info_ad_ver3_develop
    (ADID, AppID, CampaignID, ChannelID, Location, Status, Status_codekind, ADType, ADType_codekind, View_Order, 
     TotalBudget, SpentBudget, Price, webview_info, ExtractDT, trafficsourceid, Tracking_Link)
Select """+str(adid)+""" + Row_Number() Over (Order By a.TrafficsourceID) as ADID, 
       a.AppID, a.CampaignID, a.ChannelID, c.ChannelName Location,
       '01' Status,'0015' Status_codekind, '99' as ADType, '0016' as ADType_codekind,
       1 as View_Order, 0 TotalBudget, 0 as SpentBudget, 0 as Price, '' as webview_info,
       timestamp_trunc(Timestamp_Add(CURRENT_TIMESTAMP(), Interval 9 Hour),second) ExtractDT, a.TrafficSourceID, Tracking_Link
  From Tier1_BI.code_info_trafficsource_ver3_develop a
       Inner Join Tier1_BI.code_info_campaign_ver3_develop b
             ON a.campaignID = b.campaignID 
       Inner Join Tier1_BI.code_info_channel_ver3_develop c
             ON a.ChannelID = c.ChannelID And Trim(c.ChannelName) = '공통'
       Left Join Tier1_BI.code_info_ad_ver3_develop d
            ON a.AppID = d.AppID and a.CampaignID = d.CampaignID And a.ChannelID = d.ChannelID And a.TrafficSourceID = d.trafficsourceid
            And Trim(Upper(d.Location)) = Trim(Upper(c.ChannelName))
 Where d.AppID IS NULL

 
    """
    #print(query_string)
    return query_string

In [28]:
def step_06_01_query_string():
    query_string = """
Select a.AppID, a.CampaignID, a.ChannelID, a.ADID, 
        Upper(b.AppName) AppName, Upper(d.CampaignName) CampaignName, 
        Case When Upper(d.CampaignName) <> Upper('Organic') And Upper(e.channelName) = Upper('Organic') Then Upper('미입력') Else Upper(e.channelName) End channelName, 
        Case When Upper(d.CampaignName) <> Upper('Organic') And Upper(a.Location) = Upper('Organic') Then Upper('미입력') Else Upper(a.Location) End Location, 
        Case When Upper(c.schemaNM) = Upper('chacha') Then Upper('chachacha') When Upper(c.schemaNM) = Upper('pongpong') Then Upper('everypong') Else Upper(c.schemaNM) End as gamecode,
        b.GameKey, 
        Case When b.market = '02' Then Upper('iOS') Else Upper('AOS') End as OS,
        Case When Upper(g.schemaNM) = Upper('chacha') Then Upper('chachacha')
              When Upper(g.schemaNM) = Upper('pongpong') Then Upper('everypong') Else Upper(g.schemaNM) End as Channel_gamecode,
        f.Gamekey Channel_GameKey,
        d.mediatype, Upper(b.market) as market
    From Tier1_BI.code_info_ad_ver3_develop a 
        Inner Join Tier1_BI.code_info_app_ver3_develop b  ON a.AppID = b.AppID
        Inner Join Tier1_BI.code_dim_gamelist c  On Cast(b.gamekey as INT64) = c.gamekey
        Inner Join Tier1_BI.code_info_campaign_ver3_develop d  On a.CampaignID = d.CampaignID
        Inner Join Tier1_BI.code_info_channel_ver3_develop e  ON a.ChannelID = e.ChannelID
        Inner Join Tier1_BI.code_info_app_ver3_develop f  ON e.Adver_AppID = f.AppID
        Left Outer Join Tier1_BI.code_dim_gamelist g  On Cast(f.gamekey as INT64) = g.gamekey

    """
    print(query_string)
    return query_string

In [29]:
def step_06_02_query_string():
    query_string = """
        Delete From Tier1_BI.code_adkey_info_develop
        Where 1=1
        And upper(CampaignName) = upper('Netmarble Mass Marketing')
        And market not in ('01','02') 

    """
    print(query_string)
    return query_string

In [30]:
def main(meta_tuple):
    global date_key
    global query_log_path

    # Real적용시 수정
    #root_path = '/query_log/'
    root_path = '/home/mi_admin/airflow_python/query_log/'
    
    # Input Parameter
    date_key = meta_tuple

    program_name = 'tier1_bi_code_info_adver'

    # 해당테이블이 존재하는지 확인
    if globalad_exist(date_key) == 0:
        print('Tier0_BI.Log_GlobalAd_{} 테이블이 존재하지 않습니다.'.format(date_key.replace('-', '')))
        result = 'skip'
        return result
    
    
    query_log_path = root_path + program_name+'/'+datetime.now().strftime('%Y%m%d')
    ensure_dir(query_log_path)            # Directory Create
    date_key_dt = datetime.strptime(date_key, '%Y-%m-%d')
    temp_table_name = 'tier1_bi_code_info_adver_'+date_key.replace('-', '')+'_'+datetime.now().strftime('%Y%m%d%H%M%S')
    
    # 임시 고정
    #temp_table_name = 'tier1_bi_code_info_adver_20180901_112606'

    #########################################################
    #  Parameter, constant Display                          #
    #########################################################
    print('date_key        >>> {}                [{}]'.format(type(date_key), date_key))
    print('date_key_dt     >>> {}  [{}]'.format(type(date_key_dt), date_key_dt))
    print('query_log_path  >>> {}                [{}]'.format(type(query_log_path), query_log_path))
    print('temp_table_name >>> {}                [{}]'.format(type(temp_table_name), temp_table_name))
    print('')

    #########################################################
    #  Adver Code Source Data 추출                          #
    #########################################################
    step_name = '01_adver_code_source_data'
    destination_schema = 'temp'
    destination_table = temp_table_name+'_'+step_name
    source_schema = 'temp'
    source_table = temp_table_name+'_'+step_name

    query_execute_create(destination_schema, destination_table, step_01_source_data(date_key_dt), temp_table_name, step_name)

    print('*'*100)
    # DataFrome에 넣기(현재 사용안함)
    #source_df = query_execute_df(query_string)

    #########################################################
    # Campaign 생성                                         #
    #########################################################
    print('>> campaign 생성')
    campaign_schema = 'temp'
    campaign_table = temp_table_name+'_campaign'

    # 1. Source에서 Campaign관련 데이터를 테이블로 만든다.
    step_name = '02_01_campaign_groupby'
    query_execute_create(campaign_schema, campaign_table, step_02_01_query_string(source_schema, source_table), temp_table_name, step_name)

    # 2. 최종 campaignid를 추출한다.
    step_name = '02_02_get_campaignid'
    campaignid_max = get_campaignid(step_name)

    # 3. Source에서 Campagin 정보 추가하기
    step_name = '02_03_campaign_insert(source)'
    query_execute_dml(step_02_03_query_string(campaign_schema, campaign_table, campaignid_max, date_key), temp_table_name, step_name)

    # 4. 최종 campaignid를 추출한다.
    step_name = '02_04_get_campaignid'
    campaignid_max = get_campaignid(step_name)

    # 5. Netmarble Mass Marketing Campagin 정보 추가하기
    step_name = '02_05_campaign_insert(Netmarble_Mass_Marketing)'
    query_execute_dml(step_02_05_query_string(campaignid_max, date_key), temp_table_name, step_name)

    print('*'*100)

    #########################################################
    # Channel 생성                                         #
    #########################################################
    print('>> channel 생성')
    channel_schema = 'temp'
    channel_table = temp_table_name+'_channel'

    # 1. Source에서 Channel관련 데이터를 테이블로 만든다.
    step_name = '03_01_channel_groupby'
    query_execute_create(channel_schema, channel_table, step_03_01_query_string(source_schema, source_table), temp_table_name, step_name)


    # 2. 최종 channelid를 추출한다.
    step_name = '03_02_get_channelid'
    channelid_max = get_channelid(step_name)

    # 3. Source에서 Campagin 정보 추가하기
    step_name = '03_03_channel_insert(source)'
    query_execute_dml(step_03_03_query_string(channel_schema, channel_table, channelid_max, date_key), temp_table_name, step_name)

    print('*'*100)

    #########################################################
    # trafficsource 생성                                         #
    #########################################################
    print('>> trafficsource 생성')
    trafficsource_schema = 'temp'
    trafficsource_table = temp_table_name+'_trafficsource'


    # 1. Source에서 trafficsource 관련 데이터를 테이블로 만든다.
    step_name = '04_01_trafficsource_groupby'
    query_execute_create(trafficsource_schema, trafficsource_table, step_04_01_query_string(source_schema, source_table), temp_table_name, step_name)


    # 2. 최종 trafficsourceid를 추출한다.
    step_name = '04_02_get_trafficsourceid'
    trafficsourceid_max = get_trafficsourceid(step_name)

    # 3. Source에서 trafficsource 정보 추가하기
    step_name = '04_03_trafficsource_insert(source)'
    query_execute_dml(step_04_03_query_string(trafficsource_schema, trafficsource_table, trafficsourceid_max, date_key), temp_table_name, step_name)

    # 4. 최종 trafficsourceid를 추출한다.
    step_name = '04_04_get_trafficsourceid'
    trafficsourceid_max = get_trafficsourceid(step_name)

    # 5. 매스마케팅에 대한 trafficsource 정보 추가하기
    step_name = '04_05_trafficsource_insert(Netmarble_Mass_Marketing)'
    query_execute_dml(step_04_05_query_string(trafficsource_schema, trafficsource_table, trafficsourceid_max, date_key), temp_table_name, step_name)

    # 6. 최종 trafficsourceid를 추출한다.
    step_name = '04_06_get_trafficsourceid'
    trafficsourceid_max = get_trafficsourceid(step_name)

    # 7. Campaign(Publisher)에 대한 trafficsource 정보 추가하기
    step_name = '04_07_trafficsource_insert(Campaign)'
    query_execute_dml(step_04_07_query_string(trafficsource_schema, trafficsource_table, trafficsourceid_max, date_key), temp_table_name, step_name)

    print('*'*100)

    #########################################################
    # ad 생성                                         #
    #########################################################
    print('>> ad 생성')

    ad_schema = 'temp'
    ad_table = temp_table_name+'_ad'

    # 1. 최종 as를 추출한다.
    step_name = '05_01_get_adid'
    adid_max = get_adid(step_name)

    # 2. ad 정보 추가하기
    step_name = '05_02_ad_insert'
    query_execute_dml(step_05_02_query_string(source_schema, source_table, adid_max, date_key), temp_table_name, step_name)

    # 3. 최종 as를 추출한다.
    step_name = '05_03_get_adid'
    adid_max = get_adid(step_name)

    # 4. 매스마케팅에 대한 ad 정보 추가하기
    step_name = '05_04_ad_insert(Netmarble_Mass_Marketing)'
    query_execute_dml(step_05_04_query_string(ad_schema, ad_table, adid_max, date_key), temp_table_name, step_name)

    # 5. 최종 as를 추출한다.
    step_name = '05_05_get_adid'
    adid_max = get_adid(step_name)

    # 6. ad 정보 추가하기
    step_name = '05_06_ad_insert(Campaign)'
    query_execute_dml(step_05_06_query_string(ad_schema, ad_table, adid_max, date_key), temp_table_name, step_name)


    #########################################################
    # Adkey_info 생성                                       #
    #########################################################
    print('>> Adkey_info 생성')
    adkey_schema = 'Tier1_BI'
    adkey_table = 'code_adkey_info_develop'

    # 1. Source에서 Campaign관련 데이터를 테이블로 만든다.
    step_name = '06_01_adkey_info_create'
    query_execute_create(adkey_schema, adkey_table, step_06_01_query_string(), temp_table_name, step_name)

    # 2. ios, aos이외의 마켓 삭제하기
    step_name = '06_02_not_in_ios_aos_delete'
    query_execute_dml(step_06_02_query_string(), temp_table_name, step_name)
   
    
    result = 'success'
    return result

In [31]:
def parse_parameters():
    #################################################
    # 개발 완료되면 풀어야 함. !!!!!!!!!!!!!!!!!!!
    #################################################
    parser = argparse.ArgumentParser(description='시스템 구동에 필요한 파라미터를 입력하세요.')
    parser.add_argument("--date_key", help="배치작업 기준일자를 YYYY-MM-DD 형태로 입력하세요. 반드시 입력해야 하는 값입니다.", required=True)

    args = parser.parse_args()

    date_key = args.date_key

    try:
        if len(date_key.split('-')) != 3:
            raise ValueError('\'date\' parameter seems wrong. It must be in \'YYYY-MM-DD\' format.')
    except ValueError:
        raise

    print('[Start] {}'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")))        
    print('- Input parameter : --date_key = {}'.format(date_key))

    return date_key


In [32]:
if __name__ == '__main__':
    start_time = time.time()
    start_now = datetime.now()
    result = 0
    # sudo python3 tier1_bi_code_info_adver_20180903.py --date_key 2018-09-01
    result = main('2018-09-09')
    #result = main(parse_parameters())
    print('*'*100)
    print('[Complete] {} ~ {} total elapsed : {} seconds'.format(start_now.strftime("%Y-%m-%d %H:%M:%S"),datetime.now().strftime("%Y-%m-%d %H:%M:%S"), time.time()-start_time))



Select Count(*) as onoff From Tier0_BI.__TABLES__ Where table_id='Log_GlobalAd_20180909' 
date_key        >>> <class 'str'>                [2018-09-09]
date_key_dt     >>> <class 'datetime.datetime'>  [2018-09-09 00:00:00]
query_log_path  >>> <class 'str'>                [/home/mi_admin/airflow_python/query_log/tier1_bi_adver_code/20180910]
temp_table_name >>> <class 'str'>                [tier1_bi_adver_code_20180909_20180910191604]

Execute >> [start:2018-09-10 19:16:04][elapsed:00:00:07)][     57547 rows][01_adver_code_source_data] [temp.tier1_bi_adver_code_20180909_20180910191604_01_adver_code_source_data]
****************************************************************************************************
>> campaign 생성
Execute >> [start:2018-09-10 19:16:11][elapsed:00:00:03)][      1889 rows][02_01_campaign_groupby] [temp.tier1_bi_adver_code_20180909_20180910191604_campaign]
Execute >> [start:2018-09-10 19:16:15][elapsed:00:00:02)]                 [02_02_get_campaignid] (campaigni